# 如何添加消息历史记录

:::info 先决条件

本指南假定您熟悉以下概念：
- [链接 runnable](/docs/how_to/sequence/)
- [Prompt 模板](/docs/concepts/prompt_templates)
- [Chat Messages](/docs/concepts/messages)
- [LangGraph 持久化](https://langchain-ai.github.io/langgraph/how-tos/persistence/)

:::

:::note

本指南之前涵盖了 [RunnableWithMessageHistory](https://python.langchain.com/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html) 抽象。您可以在 [v0.2 文档](https://python.langchain.com/v0.2/docs/how_to/message_history/) 中访问此版本的指南。

自 LangChain v0.3 版本发布以来，我们建议 LangChain 用户利用 [LangGraph 持久化](https://langchain-ai.github.io/langgraph/concepts/persistence/) 将 `memory` 整合到新的 LangChain 应用程序中。

如果您现有的代码已经依赖于 `RunnableWithMessageHistory` 或 `BaseChatMessageHistory`，您 **无需** 进行任何更改。我们不打算在短期内弃用此功能，因为它适用于简单的聊天应用程序，并且任何使用 `RunnableWithMessageHistory` 的代码将继续按预期工作。

有关更多详细信息，请参阅 [如何迁移到 LangGraph Memory](/docs/versions/migrating_memory/)。
:::

在构建聊天机器人时，将对话状态传入和传出链至关重要。LangGraph 实现了内置的持久化层，允许链状态自动持久化到内存或外部后端，例如 SQLite、Postgres 或 Redis。详细信息可在 LangGraph [持久化文档](https://langchain-ai.github.io/langgraph/how-tos/persistence/) 中找到。

在本指南中，我们将通过将任意 LangChain runnable 包装在一个最小的 LangGraph 应用程序中来演示如何为其添加持久化。这使我们能够持久化消息历史记录和其他链状态元素，从而简化多轮应用程序的开发。它还支持多线程，使单个应用程序能够与多个用户单独交互。

## 设置

让我们初始化一个聊天模型：

import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs
  customVarName="llm"
/>

In [1]:
# | output: false
# | echo: false

# import os
# from getpass import getpass

# os.environ["ANTHROPIC_API_KEY"] = getpass()
from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(model="claude-3-haiku-20240307", temperature=0)

## 示例：消息输入

为 [聊天模型](/docs/concepts/chat_models) 添加记忆是一个简单的例子。聊天模型接受消息列表作为输入并输出消息。LangGraph 提供了一个内置的 `MessagesState`，我们可以为此目的使用。

下面，我们：
1. 将图状态定义为消息列表；
2. 向图中添加一个调用聊天模型的节点；
3. 使用内存中的 checkpointer 编译图，以在运行之间存储消息。

:::info

LangGraph 应用程序的输出是其 [状态](https://langchain-ai.github.io/langgraph/concepts/low_level/)。这可以是任何 Python 类型，但在这种情况下，它通常会是与你的可运行对象模式匹配的 `TypedDict`。

:::

In [2]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = llm.invoke(state["messages"])
    # Update message history with response:
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

当我们运行应用程序时，我们会传入一个配置 `dict`，其中指定了一个 `thread_id`。此 ID 用于区分会话线程（例如，不同用户之间的会话）。

In [3]:
config = {"configurable": {"thread_id": "abc123"}}

然后，我们可以调用该应用程序：

In [4]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

It's nice to meet you, Bob! I'm Claude, an AI assistant created by Anthropic. How can I help you today?


In [5]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Bob, as you introduced yourself at the beginning of our conversation.


请注意，状态是针对不同线程分开的。如果我们向具有新的 `thread_id` 的线程发出相同的查询，模型会表明它不知道答案：

In [6]:
query = "What's my name?"
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm afraid I don't actually know your name. As an AI assistant, I don't have personal information about you unless you provide it to me directly.


## 示例：字典输入

LangChain 的 runnables 通常通过单个`dict`参数中的不同键接受多个输入。一个常见的例子是带有多个参数的提示模板。

虽然之前我们的 runnable 是一个聊天模型，但在这里我们组合了提示模板和聊天模型。

In [7]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer in {language}."),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

runnable = prompt | llm

对于此场景，我们将图状态定义为包含这些参数（除了消息历史记录）。然后，我们以与之前相同的方式定义一个单节点图。

请注意，在下面的状态中：
- 对 `messages` 列表的更新将追加消息；
- 对 `language` 字符串的更新将覆盖该字符串。

In [8]:
from typing import Sequence

from langchain_core.messages import BaseMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated, TypedDict


# highlight-next-line
class State(TypedDict):
    # highlight-next-line
    messages: Annotated[Sequence[BaseMessage], add_messages]
    # highlight-next-line
    language: str


workflow = StateGraph(state_schema=State)


def call_model(state: State):
    response = runnable.invoke(state)
    # Update message history with response:
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [9]:
config = {"configurable": {"thread_id": "abc345"}}

input_dict = {
    "messages": [HumanMessage("Hi, I'm Bob.")],
    "language": "Spanish",
}
output = app.invoke(input_dict, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

¡Hola, Bob! Es un placer conocerte.


## 管理消息历史记录

消息历史记录（以及应用程序状态的其他元素）可以通过 `.get_state` 访问：

In [10]:
state = app.get_state(config).values

print(f'Language: {state["language"]}')
for message in state["messages"]:
    message.pretty_print()

Language: Spanish
================================ Human Message =================================

Hi, I'm Bob.
================================== Ai Message ==================================

¡Hola, Bob! Es un placer conocerte.


我们也可以通过 `.update_state` 来更新 state。例如，我们可以手动添加一条新消息：

In [11]:
from langchain_core.messages import HumanMessage

_ = app.update_state(config, {"messages": [HumanMessage("Test")]})

In [12]:
state = app.get_state(config).values

print(f'Language: {state["language"]}')
for message in state["messages"]:
    message.pretty_print()

Language: Spanish
================================ Human Message =================================

Hi, I'm Bob.
================================== Ai Message ==================================

¡Hola, Bob! Es un placer conocerte.
================================ Human Message =================================

Test


有关管理状态的详细信息，包括删除消息，请参阅 LangGraph 文档：
- [如何删除消息](https://langchain-ai.github.io/langgraph/how-tos/memory/delete-messages/)
- [如何查看和更新过去的图状态](https://langchain-ai.github.io/langgraph/how-tos/human_in_the_loop/time-travel/)